# **Web Scraping of flipkart Stationary Category**

## Listing categories available on the flipkart website

In [273]:
!pip install pandas --quiet
import pandas as pd
!pip install requests --quiet
import requests

In [274]:
response = requests.get("https://www.flipkart.com/the-stationery-store?fm=neo%2Fmerchandising&iid=M_d8a22038-d0a0-4ee3-b856-f6dfe965eff1_1_372UD5BXDFYS_MC.JNOTUFQEWK7F&otracker=hp_rich_navigation_10_1.navigationCard.RICH_NAVIGATION_Beauty%252C%2BToys%2B%2526%2BMore~Stationery%2B%2526%2BOffice%2BSupplies_JNOTUFQEWK7F&otracker1=hp_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_10_L1_view-all&cid=JNOTUFQEWK7F")
response.status_code

200

In [275]:
type(response)

requests.models.Response

In [276]:
len(response.text)

1024303

In [277]:
response.text[:1000]

'<!doctype html><html lang="en"><head><link href="https://rukminim1.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/www/linchpin/fk-cp-zion/css/app_modules.chunk.94b5e7.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/www/linchpin/fk-cp-zion/css/app.chunk.6e7580.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content="102988293558"/><meta property="fb:admins" content="658873552,624500995,100000233612389"/><meta name="robots" content="noodp"/><link rel="shortcut icon" href="https://static-assets-web.flixcart.com/www/promos/new/20150528-140547-favicon-retina.ico"/><link type="application/opensearchdescription+xml" rel="search" href="/osdd.xml?v=2"/><meta property="og:type" content="website"/><meta name="og_site_name" property="og:site_name" content="Flipkart.com"/><link rel="apple-touch-icon" sizes="57x57" href="/appl

In [278]:
page_content = response.text

In [279]:
# Saving above content in html file for reference
with open("Categories_web_page.html","w") as f:
  f.write(page_content)

### Using Beautiful Soup to parse and extract information

In [280]:
!pip install BeautifulSoup4 --upgrade --quiet
from bs4 import BeautifulSoup 

In [281]:
doc = BeautifulSoup(page_content,"html.parser")

In [282]:
Stationary_cat = doc.find_all("a",{"class" : "_2SqgSY"})

In [283]:
Stationary_cat_links = []
for i in range(len(Stationary_cat)):
  Stationary_cat_links.append(("https://www.flipkart.com" + Stationary_cat[i]["href"]+"&page={}"))
Stationary_cat_links

['https://www.flipkart.com/pens-stationery/pens/pr?sid=dgv,mqm&otracker=categorytree&page={}',
 'https://www.flipkart.com/pens-stationery/calculators/pr?sid=dgv,qtq&otracker=categorytree&page={}',
 'https://www.flipkart.com/pens-stationery/diaries-notebooks/pr?sid=dgv,gas&otracker=categorytree&page={}',
 'https://www.flipkart.com/pens-stationery/art-supplies/pr?sid=dgv,j6t&otracker=categorytree&page={}',
 'https://www.flipkart.com/pens-stationery/office-equipments/pr?sid=dgv,id7&otracker=categorytree&page={}']

##Fetching product details - pen

In [284]:
#page_content1 = response.text

In [285]:
#product_details_doc = BeautifulSoup(page_content1,"html.parser")

In [286]:
Stationary_cat_links[0]

'https://www.flipkart.com/pens-stationery/pens/pr?sid=dgv,mqm&otracker=categorytree&page={}'

In [287]:
print("1 - pens\n2 - calculators\n3 - diaries-notebooks\n4 - art-supplies\n5 - office-equipments")
a = input("Select one of the category from the above list : ")
if a == "pens":
  lnk = Stationary_cat_links[0]
elif a == "calculators":
  lnk = Stationary_cat_links[1]
elif a == "diaries-notebooks":
  lnk = Stationary_cat_links[2]
elif a == "art-supplies":
  lnk = Stationary_cat_links[3]
elif a == "office-equipments":
  lnk = Stationary_cat_links[4]
else:
  print("Enter correct name")

1 - pens
2 - calculators
3 - diaries-notebooks
4 - art-supplies
5 - office-equipments
Select one of the category from the above list : calculators


In [288]:
def OMRP(x):
  return x[1:]

In [289]:
product_name_list = []
product_MRP = []
product_avg_rating = []
links = []
pages = [a for a in range(1,3)]
for page in pages:
    response = requests.get(lnk.format(page))

    page_content1 = response.text
    product_details_doc = BeautifulSoup(page_content1,"html.parser")

    Product_name = product_details_doc.find_all("a",{"class":"s1Q9rs"})
    product_links_list = []
    for i in range(len(Product_name)):
        product_links_list.append("https://www.flipkart.com" + Product_name[i]["href"])

    for i in range(len(product_links_list)):
        response = requests.get(product_links_list[i])
        pdt_name_doc = BeautifulSoup(response.text,"html.parser")
        pdt_name = pdt_name_doc.find_all("span",{"class":"B_NuCI"})
        product_name_list.append(pdt_name[0].text)
        pdt_MRP = pdt_name_doc.find_all("div",{"class":"_30jeq3 _16Jk6d"})
        product_MRP.append(OMRP(pdt_MRP[0].text))
        pdt_avg_rating = pdt_name_doc.find_all("div",{"class":"_3LWZlK"})
        product_avg_rating.append(pdt_avg_rating[0].text)
        links.append(product_links_list[i])


In [292]:
df = pd.DataFrame([product_name_list,product_MRP,product_avg_rating,links]).T
df.columns = ["Product Name","MRP",]
df.to_csv("Product_details_final_output.csv")

Extracting name from each individual product link - 2

In [291]:
# Product_name = product_details_doc.find_all("a",{"class":"s1Q9rs"})
# product_name_list1 = []
# for i in range(len(Product_name)):
#     product_name_list1.append(Product_name[i]["title"])
# len(product_name_list1)
# a = []
# response = requests.get(Stationary_cat_links[0])
# product_details_doc = BeautifulSoup(response.text,"html.parser")
# Product_name = product_details_doc.find_all("a",{"class":"s1Q9rs"})
# for i in range(len(Product_name)):
#     a.append(Product_name[i].find_next_sibling())
# prd_desc = []
# for i in range(len(a)):
#     prd_desc.append(a[i].text)